### 트리의 앙상블
- 정형데이터에 가장 뛰어난 성능을 보이는 모델들
- 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어짐
- 앙상블 모델
    - 랜덤포레스트(Random Forest)
    - 엑스트라 트리(Extra Trees)
    - 그레디언트 부스팅(Gradient Boosting)
    - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

#### 랜덤포레스트 (Random Forest)
- 앙상블 모델 중에 가장 대표격 모델로 안정적인 성능으로 널리 사용됨
- 앙상블 모델 중에 가장 먼저 시도하는 모델
- 훈련데이터에서 과대적합되는 것을 막아줌
- 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음
- 
- 학습개념
- 각각의 결정트리를 랜덤하게 만들어서 숲을 만든다고 보면 됨
- 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후, 다시 원본 훈련데이터에 반환을 함
- 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음 (중복허용)
- 
- 부트스트랩 샘플(Bootstrap Sample)
- 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
- 샘플추출 방식
    1. 원본에서 랜덤 샘플 추출
    2. 훈련 이후 사용이 끝나면 원본에 반환
    3. 다시 원본에서 샘플 추출, 이때 중복 값 추출 될수도 있음
- 위 순서를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용하고 있음
- 
- 랜덤포레스트는 교차검증을 허용함

##### 데이터 불러오기, 전처리

In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 08_wine.csv 불러와서 훈련, 테스트데이터 만들기
wine = pd.read_csv('./data/08_wine.csv')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


##### 훈련모델 생성하기
- 랜덤포레스트 패키지 : sklearn.ensemble
- 랜덤포레스트 클래스(모델) : RandomForestClassifir
- 교차검증 패키지 : sklearn.model_selection
- 교차검증 : cross_validate()
- 교차검증 후 훈련검증결과와 테스트검증결과 확인하기

In [75]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 훈련모델
rfc = RandomForestClassifier()

# 훈련하기
rfc.fit(train_input, train_target)

# 훈련, 테스트 검증
print(rfc.score(train_input, train_target))
print(rfc.score(test_input, test_target))

0.9973316912972086
0.8732307692307693


In [6]:
# 교차검증 진행
# return_train_score : 검증결과 반환받기
scores = cross_validate(rfc, train_input, train_target, return_train_score=True, n_jobs= -1)

# 최종 훈련평가 결과 및 검증결과
print(scores)

# 최종 훈련모델의 평가점수(정확도)
print('test_score 평균 =', scores['test_score'].mean())

{'fit_time': array([0.72323346, 0.75011277, 0.72872281, 0.6461935 , 0.6847887 ]), 'score_time': array([0.04448271, 0.05617046, 0.04924107, 0.03955936, 0.05191374]), 'test_score': array([0.88410256, 0.90461538, 0.90349076, 0.89014374, 0.8788501 ]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
test_score 평균 = 0.8922405096614543


In [8]:
# 특성 중요도 조회하기
print(rfc.feature_importances_)

[0.23254574 0.49599174 0.27146251]


##### oob 기능 사용
- 훈련에 참여하지 못한 잔여샘플 사용하는 기능
- 기본은 사용안함


In [9]:
rf = RandomForestClassifier(oob_score=True,
                            n_jobs= -1,
                            random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


In [10]:
print(rf.oob_score)
print(rf.oob_score_)

True
0.8981937602627258


#### 엑스트라 트리(Extra Tree)
- 랜덤포레스트와 유사하게 작동
- 기본적으로 100개의 결정트리를 훈련함
- 랜덤포레스트와의 차이점
    - 부트스트랩 샘플링을 지원하지 않음
    - 훈련데이터 전체를 이용핳여 결정트리를 생성
    - 무작위로 트리를 분리함
- 사용되는 속성 : splitter = 'random' 무작위속성
- 장점
    - 과대적합을 막고, 검증데이터의 평가 값을 높을 수 있음
    - 특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
- 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에 속도가 느린 반면
- 엑스트라트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점
- 
- 사용패키지 : sklearn.ensemble
- 사용되는 클래스(모델) : ExtraTreesClassifier

In [12]:
# 코어 전체 사용, train 및 test 결과값 출력
# 교차검증 결과인 train 및 test 결과 확인

from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(n_jobs=-1,
                           random_state=42)

etc.fit(train_input, train_target)

print(etc.score(train_input, train_target))
print(etc.score(test_input, test_target))

0.9973316912972086
0.8738461538461538


In [15]:
from sklearn.model_selection import cross_validate

scores = cross_validate(etc, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(scores)

{'fit_time': array([0.6149838 , 0.59722304, 0.60463119, 0.55067348, 0.5437386 ]), 'score_time': array([0.06184626, 0.06408834, 0.0611968 , 0.0666244 , 0.06355667]), 'test_score': array([0.89025641, 0.8974359 , 0.89835729, 0.89117043, 0.88501027]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}


In [16]:
print(etc.feature_importances_)

[0.19271256 0.52151148 0.28577596]


#### 그레디언트 부스팅(Gradient Boosting)
- 깊이(max_depth)가 얕은 결정트리를 사용함
    - 기본적으로 amx_dpth=3 을 사용
    - 결정트리는 100개를 사용
- !!! 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 !!!
- 기존 훈련모델의 오차를 많이 보완해줌
- 성능향상을 위한 모델로 주로 사용됨
- 과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강함
- 
- 성능향상 테스트 방법
    - 결정트리의 갯수를 조절하면서 테스트 진행
    - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
    - 기본 학습률은 0.1
- 
- 단점
    - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에 훈련 속도가 느림
    - 이런 느린 속도를 개선한 모델이 히스토그램 기반 그래디언트 부스팅 모델

##### 그레디언트 부스팅 모델 생성
- 사용하는 클래스(모델) : GradientBoostingClassifier
- 객체 생성시 아무것도 안주고 seed값만 
- 교차검증시에는 train, test 결과값 출력

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

scores
print(scores['train_score'].mean(), scores['test_score'].mean())


0.8894704231708938 0.8715107671247301


In [18]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.12517641 0.73300095 0.14182264]


##### 학습률 적용하기
- 학습률이 커지면 트리 보정을 강하게 하기 때문에,
- 복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게 됨
- 학습률 : learning_rate = 0.1 기본값

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators= 500,
                                learning_rate= 0.2,
                                random_state=42)

scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

scores
print(scores['train_score'].mean(), scores['test_score'].mean())

0.9512006117505237 0.879719686200179


#### 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)
- 사용하는 패키지 : sklearn.ensemble
- 사용하는 클래스(모델) : HistGradientBoostingClassifier

In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

scores
print(scores['train_score'].mean(), scores['test_score'].mean())

0.9380129799494501 0.8805410414363187


In [37]:
# 히스토그램 기반 그레디언트 부스팅은 안되는듯

# hgb.fit(train_input, train_target)
# print(gb.feature_importances_)

In [23]:
hgb.score(test_input, test_target)

0.8584615384615385

#### 사이킷런 이외 다른 패키지에서 지원하는 그레디언트 부스팅 기능 모델들
- XGBoost

In [36]:
# conda install -c anaconda py-xgboost
from xgboost import XGBClassifier

xxgb = XGBClassifier(tree_method= 'hist', random_state= 42,)

scores = cross_validate(xxgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

scores
print(scores['train_score'].mean(), scores['test_score'].mean())

0.9614122399872658 0.8834151529510873


#### LightGBM
- 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
- 훈련 속도가 매우 빠름
- 최신 기술을 많이 적용하고 있어서, 인기가 올라가고 있음

In [33]:
# 아나콘다 사용자들 : conda install -c conda-forge lightgbm
# 파이썬 사용자들 : pip install lightgbm
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(tree_method= 'hist', random_state= 42,)

scores = cross_validate(lgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

scores
print(scores['train_score'].mean(), scores['test_score'].mean())

0.9413484712095832 0.8846461327857632


#### 연습

In [38]:
# [문제] 와인 데이터 사용
# 와인의 화학 조성을 사용하여 와인의 종류 예측(자유롭게)

# 특성 이름을 담고 있는 key 값 = feature_names
# 특성 데이터를 담고 있는 key 값 = data
# 범주 와인의 종류를 담고 있는 key 값 = target_names
# - 범주는 'class_0'과 'class_1'만 사용(0과 1로 변경하여 사용)
# - 0 = 레드와인, 1 = 화이트와인

# 알콜(alcohol)
# 말산(Malic acid)
# 회분(Ash)
# 회분의 알칼리도(Alcalinity of ash)
# 마그네슘(Magnesium)
# 총 폴리페놀(Total phenols)
# 플라보노이드 폴리페놀(Flavanoids)
# 비 플라보노이드 폴리페놀(Nonflavanoid phenols)
# 프로안토시아닌(Proanthocyanins)
# 색상의 강도(Color intens)
# 색상(Color)
# 희석 와인의 00280/00315 비율 (00280/00315 of diluted wines)
# 프롤린(Proline)

In [39]:
from sklearn.datasets import load_wine

wine_all = load_wine()
print(wine_all.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])


In [41]:
data = wine_all['data']
target = wine_all['target']

In [43]:
train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      random_state= 42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(133, 13) (133,)
(45, 13) (45,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()